# Visualizing and Understanding Neural Models in NLP  

## Abstract  

虽然神经网络已经成功应用于许多 NLP 任务，但由此产生的基于向量的模型很难解释。例如，他们如何实现组合性并不清楚，从单词和短语的含义来构建句子意义。在本文中，我们描述了用于 NLP 的神经模型中的组合性可视化的策略，其灵感来源于类似的计算机视觉工作。我们首先绘制 unit values ，用以表示negation,，强化和 concessive clauses 的组合性，从而使我们能够在 negation, 中看到众所周知的标记不对称。然后，我们介绍一些单元显著性的可视化方法，即从一阶导数对最终构成含义的贡献量。我们的通用方法在理解深度网络的组合性和其他语义属性方面可能有广泛的应用。  

## 1 Introduction  

神经模型在各种 NLP 任务中匹配或超越其他最先进系统的性能。然而，与传统的基于特征的分类器不同，这些分类器对各种可解释特征（词类，命名实体，单词形状，句法分析特征等）进行赋值和优化，深度学习模型的行为不太容易解释。深度学习模型主要通过多层神经结构对词嵌入（低维，连续，实值向量）进行操作，其每一层都被表征为一组隐藏的神经元单元。目前还不清楚深度学习模型是如何处理 composition,，实现 negation or intensification 等功能，或者结合句子不同部分的意义，建立句子意义。  

在本文中，我们探索了多种策略来解释神经模型中的意义构成。我们采用传统方法如表示性绘图，并引入简单的策略来测量神经单元对意义构成的贡献程度，它的“显著性”或使用一阶导数的重要性。  

本工作中呈现的可视化技术/模型揭示了神经模型的工作原理：例如，我们说明 LSTM 的成功是因为它能够比其他模型更加关注重要的关键词；多个composition 中的构成具有竞争性，并且这些模型能够捕捉负面的不对称，这是在自然语言理解中构成语义的重要属性；存在尖锐的空间局部性，某些维度以令人惊讶的定位方式标记否定和量化。尽管我们的尝试只涉及神经模型中的 superficial points 问题，并且每种方法都有其优缺点，但它们可以提供一些洞察神经模型在基于语言的任务中的行为，标志着在自然语言处理中理解它们如何实现有意义的 composition。  

下一节将介绍一些视觉和自然语言处理中的可视化模型，这些启发了这项工作。我们在第 3 节描述数据集和采用的神经模型。不同的可视化策略和相应的分析结果分别在第 4,5,6 节中给出，接下来是一个简短的结论。  

## 2 A Brief Review of Neural Visualization  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/2.jpg?raw=true)

通常通过将嵌入空间投影到两个维度并观察到类似的词倾向于聚集在一起，相似性通常通过图形可视化。（Karpathy et al。，2015）试图从统计观点来解释 recurrent 神经模型。其他相关尝试包括（Fyshe 等，2015；Faruqui 等，2015）。  

用于解释和可视化神经模型的方法在视觉中得到了更为重要的探索，尤其是对于卷积神经网络（CNN 或 ConvNets）（Krizhevsky等，2012），其中图像像素的原始矩阵被多层神经网络卷积并汇集到隐藏层。这些方法包括：  

（1）反演：通过训练一个额外的模型来反演表征，从不同的神经层次反馈到初始输入图像。重建背后的直觉是，可以从当前表示重构的像素内容。反演算法允许当前表示与原始图像的相应部分对齐。  

（2）反向传播和 Deconvolutional Networks：错误从输出层反向传播到每个中间层，最后传播到原始图像输入。反卷积网络以类似的方式工作，将输出逐层投影回初始输入，每一层与一个监督模型相关联，用于将较高的映射投影到较低的映射。这些策略使得有可能发现活跃区域或对最终贡献最大的区域。  

（3）生成：这组作品根据已经训练过的神经模型指导的草图生成特定类别的图像。模型以随机初始化的图像开始。在图像构建的不同阶段激活的特定图层可以帮助解释。  

尽管上述策略激发了我们在本文中提出的工作，但与 NLP 之间存在根本性差异。在 NLP 中词作为基本单位，因此（单词）向量而不是单个像素是基本单位。词的序列（例如，短语和句子）也以比像素的排列更结构的方式呈现。与我们的研究并行，Karpathy 等通过分析循环神经模型的预测和误差，从错误分析的角度探索了类似的方向。  

## 3 Datasets and Neural Models  

我们研究了两个神经模型训练的数据集，其中一个数据集规模相对较小，另一个规模较大。  

### 3.1 Stanford Sentiment Treebank  

斯坦福大学 Sentiment Treebank 是广泛用于神经模型评估的基准数据集。数据集包含每个分析树组件的 gold-standard  情绪标签，从句子到短语到单个词，用于 11855 个句子中的 215154 个短语。任务是在短语和句子两个层次上执行细粒度（非常积极，积极，中性，消极和非常消极）和粗粒度（积极与消极）分类。有关数据集的更多详细信息，请参阅 Socher et al。  

虽然这个数据集的许多研究都使用递归分析树模型，但在这项工作中，我们只使用标准序列模型（RNN 和 LSTM），因为这些模型是当前最广泛使用的神经模型，并且顺序可视化更直接。因此，我们首先将每个分析树节点转换为一个 tokens 序列。该序列首先映射到短语/句子表示并馈入 softmax 分类器。短语/句子表示使用以下三种模式构建：具有 TANH 激活函数的标准 Recurrent 序列，LSTM 和双向 LSTM。 有关这三种模型的详细信息，请参阅附录。  

### 3.2 Sequence-to-Sequence Models  

seq2seq 是旨在生成给定输入的输出文本序列的神经模型。从理论上讲，seq2seq 模型可以适应 NLP 任务，这些任务可以为给定输入的预测输出，并且由于不同的输入和输出而用于不同的目的，例如机器翻译，其输入对应于源语句和输出到目标语句；如果输入对应于消息并且输出对应于响应，则可用于对话。seq2seq 需要接受大量数据的训练，以获得学习对之间的隐式语义和句法关系。  

seq2seq 模型使用 LSTM 模型将输入序列映射到向量表示，然后基于预先获得的表示顺序预测 tokens。该模型定义了输出（Y）上的分布，并使用 softmax 函数依次预测给定输入（X）的 tokens。  

## 4 Representation Plotting  

我们从简单的 plots 开始，使用斯坦福情绪树库（Stanford Sentiment Treebank）揭示 local compositions。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/1.jpg?raw=true)

**Local Composition** 图1 显示了一个 60d 热图矢量，用于表示选定的单词/短语/句子，重点在于 extent 修改（状语和 adjectival）和 negation。短语或句子的嵌入是通过组合来自预训练模型的词表达来实现的。  

我们在图2 中使用 tsne（Van der Maaten和Hinton，2008）对单词和短语进行可视化，故意添加一些随机单词以供比较。可以看出，神经模型很好地学习了 local compositionally 性质，将否定词+积极词（“不好”，“不好”）与负面词组合在一起。还要注意否定的不对称性：“not bad”更多的和负性词在聚在一起（如图1和图2所示）。这种不对称性在语言学中得到了广泛的讨论，例如由于显著性而引起的，因为'好'是该尺度的未标记方向。这表明，虽然模型在图1 中似乎集中于某些 negation，但神经模型不仅仅是学习将固定变换应用于“不”，而是能够捕捉不同单词构成中的细微差异。  

**Concessive Sentences ** 在让步性句子中，两个从句具有相反的词性，通常与期望相反的含义相关。图3 绘制出两个让步从句随时间演变的表示图。这些图表明：  

1.对于目标是预测特定语义维度（与语言模型词预测等一般任务相反）的情感分析等任务而言，过大的维度会导致许多维度无法运行（值接近0），从而导致两个维度 相反情绪的句子仅在几个维度上有所不同。 这可以解释为什么更多的维度不一定会导致这些任务更好的表现（例如，如（Socher等人，2013）中所报道的，当词维度设置在25和35之间时达到最佳性能）。  

。。。  

**Clause Composition** 在图4 中，我们更详细地探讨了此 clause composition。通过添加负面 clauses，比如“although it had bad acting”，或者““but it is too long”到简单的积极的“I like the movie”的结尾，这些表现力更接近负面情绪区域。相比之下，为负面 clause 增加一个concessive clause 并不会朝着积极的方向发展；“I hate X but ...”仍然是非常消极的，与“I hate X”没有什么不同。这种差异再次表明该模型能够捕捉负面不对称。  

## 5 First-Derivative Saliency  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/3.jpg?raw=true)

在本节中，我们描述了另一种策略，它受到视觉中的后向传播策略的启发。它衡量每个输入单位对最终决策的贡献量，可以用一阶导数来近似。  

更正式地说，对于分类模型，输入 E 与标准类别标签 c 相关联。（根据NLP任务，输入可以是单词或单词序列的嵌入，而标签可以是 POS 标签，情感标签，预测的下一个单词索引等）给定输入词的嵌入 E 以及关联标签 c，训练后的模型将该对（E，c）与 $S-c(E)$ 分数联系起来。目标是决定 E 的哪些units 对 $S-c(E)$ 做出最重要的贡献，并因此决定是否选择类别标签 c。   

### 5.1 Results on Stanford Sentiment Treebank  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/4.jpg?raw=true)

我们首先在斯坦福树库上说明结果。我们在图5，图6 和图7 中绘制了三个句子的显著性分数，将训练的模型应用于每个句子。每行对应于表示每个维度的每个网格的对应词表示的显著性分数。这些例子是基于明确的情绪指标“hate”，这些指标给了他们所有的负面情绪。  

**“I hate the movie”** 所有这三种模式都高度赞同“hate”，并削弱了其他 tokens 的影响力。LSTM 比标准的循环模型更加关注“hate”，但双向LSTM 显示最清晰的焦点，几乎不重视“hate”以外的单词。这大概是由于 LSTM 和 Bi-LSTM 中的门结构控制信息流，使得这些体系结构更好地滤除不太相关的信息。  

**“I hate the movie that I saw last night”** 所有三种模式都指定了正确的观点。简单的循环模型在筛选出不相关的信息方面再次做得很差，对与情绪无关的单词分配过多的显著性。然而，尽管这个句子更长，但没有一个模型受到梯度消失问题的影响；“hate”的显著性在卷积操作之后仍然突出。  

**““I hate the movie though the plot is interesting”** 简单的循环模型只强调第二个 clause “the plot is interesting”，并没有对第一个 clause “I hate the movie”给予评价。这看起来可能是由渐变消失引起的，但该模型正确地将该句子分类为非常消极，表明它成功合并来自第一个否定句子的信息。我们单独测试了单个 clause “though the plot is interesting”。标准循环模型自信地将其标记为正面。因此，尽管第一个句子中的单词显著性得分较低，但简单的循环系统设法依赖该句子，并且从后面的肯定句子中淡化信息 - 尽管后面的单词具有较高的显著性分数。这说明了显著性可视化的局限性。一阶导数不能捕获所有我们想要显示的信息，也许是因为它们只是粗略地接近 individual 贡献，并且可能不足以处理高度非线性的情况。相比之下，LSTM 强调第一个 clause，大大减弱了第二个 clause 的影响，而 Bi-LSTM 侧重于“hate the movie”和“plot is interesting”。  

### 5.2 Results on Sequence-to-Sequence Autoencoder  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/6.jpg?raw=true)

图9 表示在每个时间步骤预测对应 token 方面自动编码器的显著热图。随着解码的进行，我们通过反向传播计算每个前一个词的一阶导数。每个网格对应于每个 1000 维单词向量的平均显著性值的大小。热图给出了解码过程中神经模型行为的清晰概览。观察总结如下：  

1.对于每个预测单词的时间步长，seq2seq 模型设法将单词预测返回到输入端的对应区域（自动学习对齐），例如，当需要预测的 token 为“hate”时，以 token “hate”为中心的输入区域会产生更大的影响。  

2.神经解码将先前构建的表示与当前步骤中预测的单词相结合。随着解码的进行，初始输入对解码（即源句子中的tokens）的影响逐渐减小，因为更多的预先预测的单词被编码在矢量表示中。与此同时，语言模型的影响逐渐占据主导地位：当预测单词“boring”时，模型更重视早期预测 tokens "plot" 和 "is"，但对输入中对应的区域较少关注，eg. 输入中的单词“boring “。  

## 6 Average and Variance  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/1/5.jpg?raw=true)

对于将单词嵌入视为参数以从头开始优化的设置（与使用预先训练的嵌入相反），我们提出了第二个令人惊讶的简单而直接的方法来可视化重要指标。我们首先计算句子中所有单词的单词嵌入的平均值。一个词的显著性或影响力的衡量标准是它偏离这个平均值。这个想法是，在训练过程中，模型会学习使 indicators 与 non-indicator 词不同，即使在经过多层计算之后，它们也能够脱颖而出。  

图8 显示了方差图；每个网格对应于 $\parallel e_{ij} - \frac {1}{N_s} \sum_{i' \in N_S} e_{i'j} \parallel^2$ 的值，其中 $e_{ij}$ 表示单词 i 的第 j 维的值，N 表示句子内 token 的数量。  

如图所示，基于方差的显著性度量也很好地强调了相关的情感词。该模型确实有缺点：（1）它只能用于字嵌入是学习参数的场景（2）很明显，模型能够很好地显示局部组合性。  

## 7 Conclusion  

在本文中，我们提供了几种方法来帮助可视化和解释神经模型，了解神经模型如何构成 meanings，展示 negation 的不对称性并在某些方面解释 LSTM 在这些任务中的强大性能。  

尽管我们的尝试只是触及神经模型中的表面观点，并且每种方法都有其优点和缺点，但它们可以一起提供有关神经模型在基于语言的任务中的行为的一些见解，标志着理解它们如何在自然语言处理获得意义组成的初始步骤。我们未来的工作包括使用可视化结果来执行错误分析，以及了解不同神经模型的强度限制。  

# Distributed Representations of Words and Phrases and their Compositionality  

## Abstract  

最近引入的连续 Skip-gram 模型是学习高质量 distributed vector representations(分布向量表示)的有效方法，distributed vector representations可以捕获大量精确的句法和语义关系。在本文中，我们提出了几个扩展，提高了向量的质量和训练速度。通过对 frequent words 进行二次抽样，我们获得了显著的加速，同时还学习了更多的 regular word representations(常规单词表示)。我们还提出了一个分层 softmax 的简单替代方案，称为 negative sampling (负采样)。word representations 的一个固有限制是：它们不关心词序，而且无法表示 idiomatic phrases(习惯用语)。例如，不能简单地将“Canada/加拿大”和“Air/空中”的含义组合起来得到“Canada Air/加拿大航空公司”的含义。在这个例子的启发下，我们提出了一种在文本中查找短语的简单方法，并表明学习数百万个 phrases 的 good vector representations 是可能的。  

## 1 Introduction  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/1.jpg?raw=true)

通过分组相似的单词，在向量空间中的 distributed representations 可以帮助学习算法在 NLP 任务中获得更好的表现。最早使用 word representations 可以追溯到 1986 年(Rumelhart，Hinton 和 Williams)。这个想法已经被应用于统计语言建模且取得了相当大的成功。后续工作包括应用于自动语音识别和机器翻译，以及大范围的 NLP 任务。  

最近，Mikolov 等人引入了 Skip-gram 模型，这是一种从大量非结构化文本数据中学习高质量向量表示的有效方法。与过去大部分用于学习 word vectors 的神经网络架构不同，Skip-gram 模型的训练(参见图1)不涉及密集矩阵的乘法。这使得训练非常高效：一个优化过的单机实现可以在一天内训练超过 1000 亿字。  

使用神经网络计算的 word representation 非常有趣，因为已训练的向量明确地编码了许多语言规律和模式。有点令人惊讶的是，许多这些模式可以表示为线性变换。例如，向量计算 vec("Madrid") - vec("Spain") + vec("France") 的结果比任何其他 word vector 更接近于 vec("Paris")。   

在本文中，我们提出了原始 Skip-gram 模型的几个扩展。在训练过程中，对 frequent words 进行二次采样会导致显著的加速（大约2-10倍），并提高频率较低的 word representation 的准确性。此外，我们提出了一种用于训练 Skip-gram 模型的简化 NCE(Noise Contrastive Estimation/噪声对比估计)。结果表明，与更复杂的分层 softmax 相比，它有更快的训练速度，而且 frequent words 的 vector representation 也更好。   

words representation 天生受限于 idiomatic phrases 的表示。例如，“Boston Globe/波士顿环球报”是报纸，它不是“Boston/波士顿”和“Globe/地球”的含义的自然组合。因此，用向量来表示整个短语会使 Skip-gram 模型更具表现力。其他旨在通过组合单词向量（例如递归自动编码器/recursive autoencoders）来表示句子意义的技术也将受益于使用 phrase vectors 而不是 word vectors。   

模型从基于单词扩展到基于短语模型相对简单。首先，我们使用 data-driven 的方法识别大量的短语，然后在训练过程中将短语视为单独的 tokens (标记)。为了评估短语向量的质量，我们开发了一个包含单词和短语的类比推理任务测试集。测试集中一个典型类比对是 "Montreal":"Montreal Canadiens" :: "Toronto":"TorontoMaple Leafs" 如果最靠近 vec("Montreal Canadiens") - vec("Montreal") + vec("Toronto") 的表达是 vec("Toronto Maple  Leafs")，则被认为回答正确。   

最后，我们描述了 Skip-gram 模型的另一个有趣属性。我们发现简单的向量加法通常可以产生有意义的结果。例如，vec("Russia") + vec("river") 接近  vec("Volga River")，而 vec("Germany") + vec("capital") 接近 vec("Berlin")。这种组合性表明，通过对 word vector representation 使用基本的数学运算，可以获得不明显(non-obvious)程度的语言理解。  

## 2 The Skip-gram Model  

Skip-gram 模型的训练目标是找到可用于预测句子或文档中 surrounding words 的 word representation。更正式地，给出训练词 $w_1,w_2,,...,w_T$，Skip-gram 模型的目标是最大化对数概率：   

$\frac {1}{T} \sum_{t=1}^{T} \sum_{-c \leq j \leq c, j \neq 0} \log p(w_{t+j}|w_t) \tag{1}$

其中 c 是训练上下文（可以是中心单词 $w_t$ 的一个函数）的大小。较大的 c 意味着更多的训练样本，因此可以导致更高的准确性，同时也意味着更多的训练时间。基本 Skip-gram 公式使用 softmax 函数定义 $p(w_{t+j}|w_t)$:   

$p(w_O|w_I) = \frac{\exp (v'^T_{w_O} v_{w_I})}{\sum_{w=1}^W \exp (v'^T_{w_O} v_{w_I})} \tag{2}$

其中 $v_w$ 和 $v'_w$ 分别为 w 的输入和输出向量表示，W 为词汇表中的单词数。这个公式是不切实际的，因为计算 $\Delta \log p(w_O|w_I)$ 的花费与 W 成正比，通常会达到 $10^5 - 10^7$ 的数量级。  

### 2.1 Hierarchical Softmax  

Hierarchical(分层) softmax 是完全 softmax 的近似有效计算。它首先由 Morin 和 Bengio 在神经网络语言模型的上下文中引入。它的主要优点是，不需要评估神经网络中的 W 个输出节点以获得概率分布，仅需要评估约 log2(W) 个节点。  

分层 softmax 使用二叉树表示输出层，其中 W 个字作为其叶子节点，并且对于每个节点，显式地表示其子节点的相对概率。这些定义了一个可将概率分配给单词的 random walk(随机游走)。  

更准确地说，从一条合适的路径，可以从 root 到达每个单词 w。设 $n(w,j)$ 为从 root 到单词 w 的路径上的第 j 个节点，L(w) 为该路径的长度，则 $n(w,1) = root, n(w,L(w)) = w$。另外，对每个内节点，设 ch(n) 为 n 的 arbitrary fixed child，如果 x 是 true 则 [x]=1，否则 [x]=-1。则分层softmax 将按照如下公式定义 $p(w_O|w_I)$:   

$p(w|w_I) = \prod_{j=1}^{L(w)-1} \sigma ([n(w, j + 1) = ch(n(w, j))] \cdot v'^T_{n(w,j) v_{wI}}) \tag{3}$

其中 $\sigma(x) = 1/(1+\exp(-x))$。可以证明的是 $\sum_{w=1}^W p(w|w_I) = 1$。这意味着计算 $\log p(w_O|w_I)$ 和 $\Delta \log p(w_O|w_I)$ 的消耗与 $L(w_O)$ 成正比，通常来说不超过 $\log W$。此外，不像 Skip-gram 的标准 softmax 公式会把两个表示 $v'_w$ 和 $v_w$ 分配给每个单词 w，在分层 softmax 公式中每个单词 w 有一个 $v_w$ 且二叉树的每个内部节点 n 有一个 $v'_n$。   

分层 softmax 使用的树结构对性能有相当大的影响。Mnih 和 Hinton 探索了构建树结构的一些方法以及训练时间和结果模型精度的影响。在我们的工作中，我们使用一个霍夫曼树（binary Huffman tree），因为它将短 codes 分配给高频词，从而加快了训练速度。之前已经观察到，根据出现频率组合单词可以很好的作为基于神经网络的语言模型的一种简单加速技术。  

### 2.2 Negative Sampling  

分层 softmax 的替代方案是噪声对比估计（NCE），由 Gutmann 和 Hyvarinen 引入，并由 Mnih 和 Teh 用于语言建模。NCE 认为一个好的模型应该能够通过 logistic regression 来区分数据和噪声。这类似于 Collobert 和 Weston 使用的折页损失/hinge loss，他们通过对噪声上的数据进行排名来训练模型。  

虽然 NCE 可以最大化 softmax 的对数概率，但是 Skip-gram 模型只关注学习高质量的向量表示，因此只要向量表示保持其质量，我们可以随意简化 NCE。我们通过以下公式定义 Negative Sampling（NEG）:   

$\log \sigma(v'^T_{w_O} v_{w_I}) + \sum_{i=1}^k E_{w_i - P_n(w)} [\log \sigma(-v'^T_{w_i} v_{w_I})] \tag{4}$

用于替换 Skip-gram 中的每个 $\log p(w_O|w_I)$ 项。因此，任务是使用逻辑回归来从噪声分布 $P_n(w)$ 中区分目标单词 WO，其中每个数据样本有 k 个负样本。我们的实验表明，k 值在 5-20 范围内对于小型训练数据集是有用的，而对于大型数据集，k 可以小至 2-5。负采样和 NCE 的主要区别在于 NCE 需要样本和噪声分布的数字概率，而负采样只使用样本。尽管 NCE 使 softmax 的对数概率近似最大化，但这个属性对于我们的应用并不重要。  

NCE 和 NEG 均具有作为自由参数的噪声分布 $P_n(w)$。我们发现 unigram distribution U(w) 可以提高 NCE 和 NEG 的性能。   

### 2.3 Subsampling of Frequent Words  

在非常大的语料库中，最常见的单词可能容易出现数亿次（例如“in”，“the”和“a”）。这些单词通常比罕见单词提供更少的信息价值。例如，虽然 Skip-gram 模型受益于观察“France”和“Paris”的共现，但由于观察到“France”和“the”的频繁共现，因此几乎每一个字 - 在“the”的句子中频繁发生。这个想法也可以用于相反的方向；训练数百万个例子后，频繁词的向量表示没有显著变化。  

为了解决罕见词和常见词之间的不平衡问题，我们使用了一种简单的子采样方法：训练集中的每个词 wi 按以下公式计算的概率被丢弃  

$P(w_i) = 1 - \sqrt {\frac{t}{f(w_i)}} \tag{5}$

其中 $f(w_i)$ 是单词 wi 的频率，t 是一个选定的阈值，一般在 $10^5$ 左右。我们选择这个子采样公式是因为它在保留频率排序的同时积极地抽样频率大于 t 的词。虽然这个子采样公式是启发式选择的，但我们发现它在实践中运作良好。它会加速学习，甚至可以显著提高罕见单词的学习向量的准确性，这将在以下各节中介绍。  

## 3 Empirical Results  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/2.jpg?raw=true)

在本节中，我们评估的 Hierarchical Softmax（HS），噪声对比估计，Negative Sampling 和训练单词子采样。我们使用 Mikolov 等人介绍的类比推理任务。该任务由类似于“Germany” : “Berlin” :: “France” : ? 的类比组成，通过找到一个向量 x 来解决，vec（x）的余弦距离最接近 vec(“Berlin”) - vec(“Germany”) + vec(“France”)。如果 x 是“Paris”，这个特定的例子被认为已经被正确回答。该任务有两大类：句法类比（如“quick” : “quickly” :: “slow” : “slowly”）和语义类比，如从国家到首都的关系。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/3.jpg?raw=true)

为了训练 Skip-gram 模型，我们使用了一个由各种新闻文章组成的大型数据集（一个 Google 内部数据集，有十亿字）。我们在训练数据中丢掉了所有出现少于 5 次的单词，结果产生了一个大小为 692K 的词汇。表1 中报告了各种 Skip-gram 模型对词类比测试集的性能。该表显示 Negative Sampling 在类比推理任务上优于 Hierarchical Softmax，并且性能甚至比噪声对比估计略好。频繁词的二次采样提高了几倍的训练速度，并使词表达更加准确。  

可以说，skip-gram 模型的线性使得它的向量更适合于这种线性类比推理，但 Mikolov 等人[8]也表明，随着训练数据量的增加，由标准 S 形回归神经网络（高度非线性）学习的向量显著改善了这一任务，这表明非线性模型对线性结构的 word representations 也有帮助。  

## 4 Learning Phrases  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/4.jpg?raw=true)

正如前面所讨论的，许多短语的含义并不是单个单词含义的简单组合。为了学习短语的矢量表示，我们首先找到经常出现在一起的单词，并且很少出现在其他上下文中。 例如，“纽约时报”和“多伦多枫叶队”在训练数据中被独特的 tokens 所取代，而“this is”这个 bigram 将保持不变。  

这样，我们可以形成很多合理的短语，而不会大大增加词汇的大小；从理论上讲，我们可以使用所有的 n 元语言来训练 Skip-gram 模型，但是这会占用过多的内存。之前已经开发了许多技术来识别文本中的短语；然而，我们的工作范围不在于比较它们。我们决定使用一种简单的数据驱动方法，在这种方法中，短语是基于 unigram 和 bigram 计数形成的  

$score(w_i,w_j) = \frac {count(w_i,w_j) - \delta}{count(w_i) \times count(w_j)} \tag{6}$

δ被用作 discounting 系数，并且防止形成包含非常罕见单词的太多短语。然后将具有高于所选阈值的分数用作短语。通常，我们在阈值下降的情况下对训练数据运行 2-4 次，允许形成由多个单词组成的更长的短语。我们使用涉及短语的新类比推理任务来评估短语表示的质量。表 2 显示了这个任务中使用的五类比喻的例子。该数据集可在 Web2 上公开获得。  

### 4.1 Phrase Skip-Gram Results  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/5.jpg?raw=true)

从与之前实验相同的新闻数据开始，我们首先构建了基于短语的训练语料库，然后我们使用不同的超参数训练了多个 Skip-gram 模型。与之前一样，我们使用矢量维度 300 和上下文大小 5。此设置已经在短语数据集上实现了良好的性能，并且允许我们快速比较 Negative Sampling 和 Hierarchical Softmax，无论是否使用 frequent tokens 的 subsampling。结果总结在表3 中。  

结果表明，虽然负采样在 k = 5 时仍能达到可观的精度，但使用 k = 15 可以获得相当好的性能。令人惊讶的是，虽然我们发现 Hierarchical Softmax 在没有二次采样的情况下训练时性能较低，但是当我们对频繁字进行下采样时，它成为了表现最佳的方法。这表明子采样可以导致更快的训练，并且还可以提高准确性，至少在某些情况下。  

为了最大化词组类比任务的准确性，我们通过使用大约 330 亿字的数据集来增加训练数据量。我们使用 hierarchical softmax，维度为 1000，整个句子为上下文。这导致了一个达到 72% 准确度的模型。当我们将训练数据集的大小减少到 6B 个单词时，我们的准确率降低了 66%，这表明大量的训练数据是至关重要的。  

为了进一步深入了解不同模型所表现出来的表征之间的差异，我们使用各种模型手动检查了不常见短语的最近邻居。在表4 ，我们展示了这种比较的一个样本。 与以前的结果一致，似乎短语的最佳表示是通过具有 hierarchical softmax 和二次采样的模型学习的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/6.jpg?raw=true)

## 5 Additive Compositionality  

我们证明了由 Skip-gram 模型学习的单词和短语表示呈现出一种线性结构，可以使用简单的矢量算术执行精确的类比推理。有趣的是，我们发现 Skip-gram 表示展示了另一种线性结构，它可以通过 element-wise 方式添加它们的向量来表示有意义地组合词语。表5 说明了这种现象。  

通过检查训练目标可以解释向量的加性特性。单词矢量与 softmax 非线性的输入成线性关系。当词向量被训练来预测句子中的周围词时，向量可以被看作表示词出现的上下文的分布。这些值与由输出层计算的概率以对数形式相关，所以两个单词向量的总和与两个上下文分布的乘积有关。该乘积在这里用作 AND 函数：由两个单词向量分配高概率的单词将具有高概率，而其他单词将具有低概率。因此，如果“伏尔加河”与“俄罗斯”和“河流”一起经常出现在同一个句子中，这两个单词向量的总和将导致这样一个特征向量接近“伏尔加河”向量，。  

## 6 Comparison to Published Word Representations  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/2/7.jpg?raw=true)

许多以前从事基于神经网络词汇表征的作者已经发表了他们的结果模型供进一步的使用和比较：其中最着名的作者是 Collobert 和 Weston [2]，Turian ，Mnih 和 Hinton [10]。我们从 web3 下载了他们的单词向量。Mikolov 等人[8]已经评估了单词类比任务中的这些单词表示，其中 Skip-gram 模型以巨大的优势实现了最佳性能。  

为了更深入地了解学习向量质量的差异，我们通过显示表6 中不常用词的最近邻居来提供经验性比较。这些例子表明，在大型语料库上训练的 Skip-gram 模型明显优于所有其他模型。这部分可以归因于这个模型已经被训练了大约 300 亿个单词，这比以前的工作中使用的典型大小多出大约两到三个数量级的数据。有趣的是，尽管训练集更大，但 Skip-gram 模型的训练时间仅仅是以前模型架构所需的时间复杂度的一小部分。  

## 7 Conclusion  

这项工作有几个关键贡献。我们展示了如何使用 Skip-gram 模型来训练单词和短语的分布式表示，并且证明这些表示呈现出可以进行精确的类比推理的线性结构。本文介绍的技术也可用于训练[8]中介绍的 bag-of-words 模型。  

由于采用了计算效率高的模型架构，我们成功地对比以前发布的模型多数量级的数据进行了模型训练。这导致学习的单词和短语表示的质量大大提高，特别是对于罕见的实体。我们还发现，频繁词汇的二次抽样导致更快的训练和更好地表达不常见的单词。我们的论文的另一个贡献是负采样算法，它是一种非常简单的训练方法，可以学习精确的表示，特别是对于频繁的词。  

训练算法和超参数选择的选择是一个任务特定的决定，因为我们发现不同的问题有不同的最优超参数配置。在我们的实验中，影响性能的最关键的决定是模型架构的选择，矢量的大小，子采样率以及训练窗口的大小。  

这项工作的一个非常有趣的结果是单词向量可以通过简单的向量添加而有意义地组合。学习本文中表达的短语的另一种方法是简单地用单个标记表示短语。 这两种方法的组合提供了一种强大而简单的方法，即如何表示更长的文本片段，同时具有最小的计算复杂度。因此，我们的工作可以被看作是试图用递归矩阵向量操作来表示短语的现有方法的补充[16]。  

我们根据本文描述的技术制定了用于训练词和短语向量的代码，可作为开源项目使用。

# Word2Vec Tutorial  

本教程介绍 Word2Vec 的 skip-gram 神经网络架构。本教程的目的是跳过关于 Word2Vec 的常见介绍性和抽象的见解，并深入了解更多细节。具体来说，我正在 diving into 神经网络模型。  

## The Model  

skip-gram 神经网络模型其最基本的形式实际上是非常简单的。

让我们从关于我们要做什么的高层次洞察开始。Word2Vec 使用了一个你可能在机器学习中看到过的技巧。我们将训练一个带有单个隐藏层的简单神经网络来完成特定的任务，但是我们实际上并没有将这个神经网络用于我们训练的任务！相反，目标实际上只是学习隐藏层的权重 - 我们会看到这些权重实际上是我们试图学习的“单词向量”。  

另一个你可能见过这个技巧的地方是无监督的特征学习，在这里你训练一个自动编码器来压缩隐藏层中的输入矢量，并将其解压缩回输出层的原始数据。 训练完成后，您会剥离输出层（解压缩步骤），然后使用隐藏层 - 这是学习良好图像特征而不标记训练数据的诀窍。

## The Fake Task  

所以现在我们需要谈论这个我们将要建立神经网络来执行的“假”任务，然后我们会稍后再回到这个间接给我们提供那些我们真正需要单词向量的方法。

我们将训练神经网络来完成以下工作。给定一个句子中的特定单词（输入单词），查看附近的单词并随机选择一个单词。网络将告诉我们在我们的词汇中每个词是我们选择的“附近词”的可能性。

当我说“附近”时，算法实际上有一个“窗口大小”参数。一个典型的窗口大小可能是 5，意味着前后各有 5 个字（总共10个字）。
输出概率将与在我们的输入词附近找到每个词汇单词的可能性有关。例如，如果你给训练好的网络输入单词“苏联”，输出概率对于“联盟”和“俄罗斯”这样的单词来说要比“西瓜”和“袋鼠”这样无关的单词要高得多。

我们将通过输入我们的训练文档中找到的单词对来训练神经网络。下面的例子显示了一些训练样本（单词对），我们将从“The quick brown fox jumps over the lazy dog”一句中取出。我已经使用了一个 2 的小窗口大小作为例子。以蓝色突出显示的单词是输入单词。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/1.jpg?raw=true)

网络将从每次配对显示的次数中学习统计数据。例如，网络可能会获得更多（“苏联”，“联盟”）的训练样本，而不是（“苏联”，“萨斯奎奇”）的训练样本。训练结束后，如果您输入“苏联”一词作为输入，那么它将输出“联盟”或“俄罗斯”的概率高于“Sasquatch”的概率。  

## Model Details  

那么这是如何表现的？

首先，你知道你不能将一个单词作为一个文本字符串输入到神经网络，所以我们需要一种方法来将这些单词表达给网络。为此，我们首先根据训练文档建立一个单词词汇表 - 假设我们有一个包含 10,000 个独特词汇的词汇表。

我们将代表像“蚂蚁”这样的输入词作为一个 one-hot vector。这个矢量将有 10,000 个 components，我们将在对应于单词“蚂蚁”的位置放置一个“1”，并在所有其他位置放置一个“0”。

网络的输出是一个单一的矢量（也有 10,000 个 components），对于我们的词汇表中的每个单词，它表示该单词是输入单词附近词的概率。

这是我们神经网络的架构。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/2.jpg?raw=true)

隐层神经元没有激活函数，但输出神经元使用 softmax。我们稍后再回来。

当在单词对上训练这个网络时，输入是代表输入词的 one-hot 矢量，并且训练输出也是代表输出词的 one-hot 矢量。但是，当你在输入字上评估训练好的网络时，输出矢量实际上是一个概率分布（即一组浮点值，而不是一个 one-hot 矢量）。  

## The Hidden Layer  

对于我们的例子，我们要说我们正在学习 300 个特征的单词向量。因此，隐藏层将由一个 10,000 行（我们词汇表中的每个词）和 300 列（每个隐藏的神经元）的权重矩阵表示。

300 特征是 Google 在他们发布的 Google 新闻数据集训练模型中使用的特征数量。特征的数量是一个“超参数”，你只需要调整你的应用程序（即尝试不同的值，看看什么会产生最好的结果）。

如果你看这个权重矩阵的行，这些实际上是我们的词向量！  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/3.jpg?raw=true)

因此，所有这些的最终目标只是为了学习这个隐藏层的权重矩阵 - 当我们完成时我们会抛弃输出层！

但是，让我们回过头来看看我们要训练的这个模型的定义。

现在，你可能会问自己 - “这个 one-hot 矢量几乎全是零...那有什么效果呢？”如果你将一个 1×10,000 one-hot 矢量乘以一个 10,000×300 的矩阵，它将有效地选择矩阵行对应于“1”。这里有一个小例子给你一个视觉效果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/4.jpg?raw=true)

这意味着此模型的隐藏层实际上只是作为查找表来操作。隐藏层的输出只是输入单词的“单词向量”。  

## The Output Layer  

“蚂蚁”的 1×300 word 矢量然后被送到输出层。输出层是一个 softmax 回归分类器。这里有一个关于 Softmax 回归的深入教程，但其要点是每个输出神经元将产生一个介于 0 和 1 之间的输出，并且所有这些输出值的总和将加起来是 1。

具体来说，每个输出神经元都有一个权重向量，它与来自隐藏层的单词向量相乘，然后将函数 exp(x) 应用于结果。最后，为了使输出总和为 1，我们将这个结果除以所有 10,000 个输出节点的结果之和。

下面是计算单词“car”的输出神经元输出的图示。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/5.jpg?raw=true)

请注意，神经网络不知道输出词相对于输入词的偏移量。它并没有为输入之前的单词与之后的单词学习不同的一组概率。为了理解这个含义，假设在我们的训练语料库中，单词 'York' 的每一个单词前面都有 'New' 单词。也就是说，至少根据训练数据，'New' 将在 'York' 附近有 100% 的概率。然而，如果我们把 'York' 附近的 10 个单词随机选取其中的一个，那么 'New' 的概率不是100%。你可能选择了附近的其他单词之一。  

## Intuition  

好的，你准备好了解这个网络的一些情况吗？

如果两个不同的单词具有非常相似的“contexts”（也就是说，他们周围可能出现哪些单词），那么我们的模型需要为这两个单词输出非常相似的结果。 对于这两个词，网络输出类似上下文预测的一种方式是如果词向量相似。所以，如果两个词有相似的上下文，那么我们的网络就有动机学习这两个词的相似词向量！

两个词有相似的 contexts 是什么意思？ 我认为你可以期望像“智能”和“聪明”这样的同义词具有非常相似的 contexts。或者，与“引擎”和“传播”相关的词语可能也有类似的语境。

这也可以处理词干 - 网络可能会学习单词“ant”和“ants”的相似单词向量，因为它们应该有类似的上下文。  

## part 2

在 word2vec 教程的第 2 部分中，我将介绍对基本 skip-gram 模型的一些额外修改，实际上这些修改对于使其可以有效训练非常重要。

当您阅读 Word2Vec 的 skip-gram 模型教程时，您可能已经注意到了一些东西 - 这是一个巨大的神经网络！

在我给出的例子中，我们有 300 个 components 的单词向量和 10,000 个单词的词汇表。回想一下，神经网络有两个权重矩阵 - 隐藏层和输出层。这两层都会有一个权重矩阵，每个权重矩阵为 300 x 10,000 = 300 万！

在一个大的神经网络上运行梯度下降是很缓慢的。更糟糕的是，你需要大量的训练数据才能调整许多权重并避免过拟合。数百万的权重乘以数十亿训练样本意味着训练这个模型将成为一个野兽。

Word2Vec 的作者在他们的第二篇论文中讨论了这些问题。

第二篇论文有三个创新点：

- 1 在他们的模型中将常用单词对或短语视为单个“单词”。  

- 2 对频繁的词进行子采样以减少训练实例的数量。  

- 3 用他们称之为“负采样”的技术修改优化目标，这会使每个训练样本只更新一小部分模型的权重。

值得注意的是，对频繁词进行二次抽样和应用负抽样不仅减少了训练过程的计算负担，而且还提高了其结果词向量的质量。

## Word Pairs and “Phrases”  

作者指出，像“Boston Globe”（一家报纸）这样的词组与“Boston”和“Globe”这两个单词有着非常不同的含义。因此，将“Boston Globe”无论出现在文本中的任何地方都视为具有自己词汇向量表示的单个词汇，这是有道理的。

您可以在他们发布的模型中查看结果，该结果从 Google 新闻数据集中对 1000 亿字进行了训练。在模型中加入短语使词汇量增加到 300 万字！

如果你对他们所产生的词汇感兴趣，我可以稍微探讨一下。你也可以在这里浏览他们的词汇。

短语检测包含在他们的论文的“Learning Phrases”部分。他们在 word2phrase.c 中分享了他们的实现。

我不认为他们的短语检测方法是他们论文的一个关键贡献，但是我会分享一些，因为它非常简单。

他们的工具每次只查看 2 个单词的组合，但可以多次运行以获得更长的短语。因此，第一遍会选择“New_York”，然后再次运行它 将 “New_York_City” 选为“New_York”和“City”的组合。

该工具计算训练文本中两个单词的每个组合出现的次数，然后将这些计数用于等式中以确定将哪些单词组合成短语。它还喜欢使用不常见字组成的短语词组，以避免 “and the” 或“this is”等常见词组成词组。

我认为有一种替代词组识别策略，就是将所有维基百科文章的标题用作词汇。

## Subsampling Frequent Words  

在本教程的第1 部分中，我展示了如何从源文本创建训练样本，但我会在此重复。下面的例子显示了一些训练样本（单词对），我们将从“The quick brown fox jumps over the lazy dog”一句中取出。我已经使用了一个 2 的小窗口作为例子。以蓝色突出显示的单词是输入单词。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/OCR/3/1.jpg?raw=true)

用“the”这个常见词有两个“问题”：

- 1 当看单词对时，(“fox”，“the”)并没有告诉我们很多关于“fox”的含义。“the”几乎出现在每个单词的上下文中。  

- 2 我们将有比我们需要为“the”学习一个好的向量的更多的样本(“the”，...)。  

Word2Vec 实现了一个“subsampling”方案来解决这个问题。对于我们在训练文本中遇到的每个单词，我们都有可能将其有效的从文本中删除。删除单词的概率与单词的频率有关。

如果我们的窗口大小为 10，并且从文本中删除“the”的特定实例：

- 1 当我们训练剩余的单词时，“the”不会出现在任何上下文窗口中。  

- 2 当“the”是输入词时，我们将减少 10 个训练样本。

请注意这两个效果如何帮助解决上述两个问题。

## Sampling rate  

word2vec C 代码实现了计算词汇表中给定单词的概率的等式。

wi 是单词，z（wi）是语料库中所有单词的分数。例如，如果单词“peanut”在 10 亿字的语料库中出现 1000 次，则 z（'peanut'）= 1E-6。

代码中还有一个 'sample' 参数，用于控制发生多少次采样，默认值为 0.001。较小的“sample”意味着 words 更难以保留。

P（wi）是保留单词的概率：

$P(w_i) = (\sqrt {\frac {z(w_i)}{0.001}} + 1) \cdot \frac {0.001}{z(w_i)}$

您可以在Google中快速绘制此图形以查看形状。  

没有一个单词应该是语料库中非常大的比例，所以我们希望在 x 轴上查看相当小的值。

这里有一些有趣的地方在这个函数（也是使用默认的样本值0.001）。

- 当 z（wi）≤ 0.0026 时，P（wi）= 1.0（保持100%的概率）。这意味着只有占总词数超过 0.26% 的词将被二次抽样。  

- 当 z（wi）= 0.00746 时，P（wi）= 0.5（保持50%的机会）。  

- 当z（wi）= 1.0，P（wi）= 0.033（保持3.3%的机会）。也就是说，如果语料库完全由 wi 词组成，这当然是荒谬的。  

你可能会注意到这篇论文定义的这个函数与 C 代码中实现的有些不同，但我认为 C 实现是更权威的版本。  

## Negative Sampling  

训练一个神经网络意味着输入一个训练样本就稍微调整所有的神经元权重，以便更准确地预测训练样本。换句话说，每个训练样本都会调整神经网络中的所有权重。

正如我们上面所讨论的那样，我们的单词词汇的大小意味着我们的 skip-gram 神经网络有很多权重，所有这些都将被我们数十亿训练样本中的每一个稍微更新！

负采样通过让每个训练样本只修改一小部分权重而不是全部权重来解决这个问题。这是它的工作原理。

当在单词对(“fox”, “quick”)上训练网络时，请回想一下，网络的“标签”或“正确输出”是一个 one-hot 向量。也就是说，对于对应于“quick”的输出神经元输出 1，并且对于所有其他数千个输出神经元输出 0。

对于 negative  抽样，我们将随机选择一小部分“negative ”单词（比如 5）来更新权重。（在这种情况下，“negative ”一词是我们希望网络输出 0 的那个词）。我们还将更新我们的“positive”单词的权重（在我们当前的例子中，这是“quick”一词）。

该论文指出，选择 5-20 个单词适用于较小的数据集，并且对于大型数据集只能使用 2-5 个单词。  

回想一下，我们模型的输出层有一个 300 x 10,000 的权重矩阵。所以我们只是更新我们正面词的权重（“quick”），加上我们想要输出的其他 5 个词的权重。这总共有 6 个输出神经元，总共有 1,800 个权重值。这只是输出层 3M weights 的 0.06%！

在隐藏层中，只更新输入词的权重（无论您是否使用 Negative 采样，情况都是如此）。

## Selecting Negative Samples  

“负样本”（即我们将训练输出为 0 的 5 个输出字）是使用“unigram distribution”选择的。

本质上，选择一个词作为负样本的概率与其频率有关，更频繁的词更可能被选作负样本。

在 word2vec C 实现中，您可以看到这个概率的等式。每个单词的权重等于其频率（字频）的 3/4 次方。选择一个词的概率只是它的权重除以所有单词的权重之和。

$P(w_i) = \frac {f(w_i)^{3/4}}{\sum_{j=0}^{n}(f(w_i)^{3/4})}$

将频率提高到 3/4 power 的决定似乎是经验性的；在他们的论文中他们说它超越了其他数字。您可以查看函数的形状 - 只需在Google中输入：“plot y = x^(3/4) and y = x”，然后放大范围 x = [0，1]。它有一个略微增加的曲线。

这种选择在 C 代码中实现的方式很有趣。他们有一个拥有 100M 元素的大array （他们称之为 unigram table）。他们多次使用用词汇表中每个单词的索引填充这个表格，单词索引出现在表格中的次数由 $P(w_i) * table_size$ 给出。然后，为了实际选择一个负样本，您只需生成一个介于 0 到 100M 之间的随机整数，然后在表格中使用该索引处的单词。由于较高的概率词在表格中出现次数更多，因此您更有可能选择这些词语。  

## 参考  

- 1 [Visualizing and Understanding Neural Models in NLP](https://arxiv.org/abs/1506.01066)  
- 2 [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)  
- 3 [Word2Vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  